This file contains the calculation of efficiency histogram which will also serve as an input to LAURA++. The parallelization strategy of the dataframe is necessary as we need to generate a lot of generator level events to lower the uncertainity on the efficiency (and also so that the generated events cover the entire DP, so we could achieve a much finer binning)

In [ ]:
import uproot 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import dask.dataframe as dd
import ROOT
import os
import numpy as np
from matplotlib.colors import LogNorm
from scipy.interpolate import RectBivariateSpline

In [ ]:
file = uproot.open("generated-output.root")
tree = file["tree;1"]
columns_to_read = ['isSignal', 'B1_deltaE', 'M', 'Dplus_M','__event__', '__production__',
                  '__ncandidates__','B1_mcPX', 'B1_mcPY', 'B1_mcPZ', 'B1_mcE',
                  'pim11_mcPX', 'pim11_mcPY', 'pim11_mcPZ', 'pim11_mcE',
                  'pim12_mcPX', 'pim12_mcPY', 'pim12_mcPZ','pim12_mcE',
                  'Dplus_mcPX', 'Dplus_mcPY', 'Dplus_mcPZ','Dplus_mcE',
                  'pim12_M','pim11_M','B1_M']
df = tree.arrays(expressions=columns_to_read, library="pd")

In [ ]:
# Random best candidate selection 
# Parallelize the dataframe using dask and divide the dataframes into two dataframes, one with 2 candidates and one with 1 candidate.

df = dd.from_pandas(pd.concat([df]), npartitions=20)
 
df_filtered = df[df['__ncandidates__'] == 2]

grouped = df_filtered.groupby(['__event__', '__production__'])

def select_random_candidate(group):
    return group.sample(frac=1).head(1)

random_candidates = grouped.apply(select_random_candidate,meta = df_filtered.dtypes.to_dict()).reset_index(drop=True)

df_single_candidates = df[df['__ncandidates__'] == 1]

final = dd.concat([random_candidates, df_single_candidates])

df_1 = final.compute()  


In [ ]:
import time 

start_time = time.time()

def calculate_momentum_partition(df):
    # Initialize empty lists to store the boosted momenta
    boosted_pim11_p = []
    boosted_pim12_p = []

    for index, row in df.iterrows():
        # Create TLorentzVector for the B meson
        vector_B_meson = ROOT.TLorentzVector(row['B1_mcPX'], row['B1_mcPY'], row['B1_mcPZ'], row['B1_mcE'])

        # Create TLorentzVector for pim11 and pim12
        vector_pim11 = ROOT.TLorentzVector(row['pim11_mcPX'], row['pim11_mcPY'], row['pim11_mcPZ'], row['pim11_mcE'])
        vector_pim12 = ROOT.TLorentzVector(row['pim12_mcPX'], row['pim12_mcPY'], row['pim12_mcPZ'], row['pim12_mcE'])

        # Calculate the boost vector for the B meson's rest frame
        boost_vector = -vector_B_meson.BoostVector()

        # Boost pim11 and pim12 to the B meson's rest frame
        vector_pim11.Boost(boost_vector)
        vector_pim12.Boost(boost_vector)

        # Extract the boosted momenta and append to the lists
        boosted_pim11_p.append(vector_pim11.P())
        boosted_pim12_p.append(vector_pim12.P())

    # Create a new DataFrame with the boosted momenta
    result_df = pd.DataFrame({'boosted_pim11_p': boosted_pim11_p, 'boosted_pim12_p': boosted_pim12_p})
    return result_df


#Original dataframe to dask df
ddf = dd.from_pandas(df_1, npartitions=10)

# Apply the function to each partition of the dask df
momenta = ddf.map_partitions(calculate_momentum_partition, meta={'boosted_pim11_p': 'float64', 'boosted_pim12_p': 'float64'})

momenta_reset = momenta.reset_index(drop=True)
ddf_reset = ddf.reset_index(drop=True)

# Now concatenate along columns
A = dd.concat([momenta_reset, ddf_reset], axis=1)

# Calculating DP variables
A['mdpluspim112'] = (A['Dplus_mcE'] + A['pim11_mcE'])**2 - (A['Dplus_mcPX'] + A['pim11_mcPX'])**2 - (A['Dplus_mcPY'] + A['pim11_mcPY'])**2 - (A['Dplus_mcPZ'] + A['pim11_mcPZ'])**2
A['mdpluspim122'] = (A['Dplus_mcE'] + A['pim12_mcE'])**2 - (A['Dplus_mcPX'] + A['pim12_mcPX'])**2 - (A['Dplus_mcPY'] + A['pim12_mcPY'])**2 - (A['Dplus_mcPZ'] + A['pim12_mcPZ'])**2
A['mpim11pim122'] = (A['pim11_mcE'] + A['pim12_mcE'])**2 - (A['pim11_mcPX'] + A['pim12_mcPX'])**2 - (A['pim11_mcPY'] + A['pim12_mcPY'])**2 - (A['pim11_mcPZ'] + A['pim12_mcPZ'])**2

final_ddf = A[['boosted_pim11_p', 'boosted_pim12_p', 'mdpluspim112', 'mdpluspim122','mpim11pim122','M','pim12_M','pim11_M','B1_M','Dplus_M',
               'Dplus_mcPX','Dplus_mcPY', 'Dplus_mcPZ','Dplus_mcE',
               'pim11_mcPX', 'pim11_mcPY', 'pim11_mcPZ','pim11_mcE',
               'pim12_mcPX', 'pim12_mcPY', 'pim12_mcPZ','pim12_mcE']]

def swap_columns_in_partition(df):
    # Identify columns starting with 'mdpluspim112' and 'mdpluspim122'
    pim11_cols = [col for col in df.columns if col.startswith('mdpluspim112')]
    pim12_cols = [col for col in df.columns if col.startswith('mdpluspim122')]
    
    # Iterate over the DataFrame rows
    for i, row in df.iterrows():
        # Check the condition to swap values
        if row['boosted_pim12_p'] > row['boosted_pim11_p']:
            for pim11_col, pim12_col in zip(pim11_cols, pim12_cols):
                # Swap the values
                df.at[i, pim11_col], df.at[i, pim12_col] = row[pim12_col], row[pim11_col]
                
    return df

sorted_final = final_ddf.map_partitions(swap_columns_in_partition, meta=final_ddf)

B = sorted_final.compute()

def helicity_angle(row):
    # Create TLorentzVector for particles D, B (pi_minus_1), and C (pi_minus_2)
    vector_D = ROOT.TLorentzVector(row['Dplus_mcPX'], row['Dplus_mcPY'], row['Dplus_mcPZ'], row['Dplus_mcE'])
    vector_B = ROOT.TLorentzVector(row['pim11_mcPX'], row['pim11_mcPY'], row['pim11_mcPZ'], row['pim11_mcE'])
    vector_C = ROOT.TLorentzVector(row['pim12_mcPX'], row['pim12_mcPY'], row['pim12_mcPZ'], row['pim12_mcE'])

    # Calculate the combined momentum of the pi-p- system (vector_B + vector_C)
    vector_pi_pi = vector_B + vector_C

    # Boost vectors D, B, and C into the π−π− rest frame
    boost_vector = -vector_pi_pi.BoostVector()  # Negative because we want the boost in the opposite direction
    vector_D.Boost(boost_vector)
    vector_B.Boost(boost_vector)
    vector_C.Boost(boost_vector) 

    # Calculate the helicity angle, which is the angle between the D meson and one of the pions in the π−π− rest frame
    cos_theta = vector_D.Vect().Dot(vector_C.Vect()) / (vector_D.Vect().Mag() * vector_C.Vect().Mag())
    theta = ROOT.TMath.ACos(cos_theta)
    
    # Return the angle in radians divided by pi
    return (1/ROOT.TMath.Pi())*theta 

B['mpim11pim12min'] = B['pim12_M'] + B['pim11_M']
B['mpim11pim12max'] = B['B1_M'] - B['Dplus_M']
B['thprime'] = B.apply(helicity_angle, axis=1)
B['mprime'] = (1/np.pi)*np.arccos(((2*(np.sqrt(B['mpim11pim122']) - B['mpim11pim12min']))/(B['mpim11pim12max'] - B['mpim11pim12min']))-1)

end_time = time.time()

total_time = end_time - start_time
print(f"Total time taken: {total_time} seconds")

Now, we can calculate the efficiency plot and interpolate it.

In [ ]:
bina = 10
bins = [bina, bina]  
x_range = [0, 1]
y_range = [0, 0.5]

hist_A, xedges, yedges = np.histogram2d(rec['mprime'], rec['thprime'], bins=bins, range=[x_range, y_range])
hist_nores, _, _ = np.histogram2d(gen['mprime'], gen['thprime'], bins=bins, range=[x_range, y_range])

# Ratio calculation
ratio = np.divide(hist_A, hist_nores, out=np.zeros_like(hist_nores), where=(hist_nores != 0))
ratio[(hist_nores == 0) & (hist_A == 0)] = 0  

# Midpoints for interpolation
x_mid = 0.5 * (xedges[:-1] + xedges[1:])
y_mid = 0.5 * (yedges[:-1] + yedges[1:])

# Interpolation setup
spline = RectBivariateSpline(x_mid, y_mid, ratio.T, kx=3, ky=3)

X_fine = np.linspace(x_range[0], x_range[1], 400)
Y_fine = np.linspace(y_range[0], y_range[1], 400)
X_fine_grid, Y_fine_grid = np.meshgrid(X_fine, Y_fine)
ratio_fine = spline.ev(X_fine_grid.ravel(), Y_fine_grid.ravel()).reshape(400, 400)


# Plotting
plt.figure(figsize=(12, 7))
ax = plt.gca()
img = ax.imshow(ratio_fine.T, extent=(x_range[0], x_range[1], y_range[0], y_range[1]), origin='lower', aspect='auto', cmap='plasma', norm=LogNorm())
cbar = plt.colorbar(img, label='Efficiency')
cbar.ax.set_ylabel('Efficiency', rotation=270, labelpad=20, fontsize=20)
cbar.ax.tick_params(labelsize=18)
ax.set_xlabel(r'$m^{\prime}$', fontsize=18)  
ax.set_ylabel(r'$\theta^{\prime}$', fontsize=18)
ax.minorticks_on()
ax.tick_params(axis='both', which='major', labelsize=14)
ax.tick_params(axis='both', which='minor', labelsize=10)
ax.grid(False)
plt.savefig('efficieincy_logscale.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
#Save the histogram to a ROOT file.
file = ROOT.TFile("efficiency.root", "RECREATE")
hist_root = ROOT.TH2F("h_ratio", "Efficiency;M_{D^+ \pi_{slow}^-}^{2} [GeV^2];M_{D^+ \pi^-}^{2} [GeV^2]", bins[0], x_range[0], x_range[1], bins[1], y_range[0], y_range[1])

for i in range(bins[0]):
    for j in range(bins[1]):
        hist_root.SetBinContent(i + 1, j + 1, ratio_fine[i][j])

hist_root.Write()
file.Close()